In [1]:
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
import math
import json

from sklearn.model_selection import *
from sklearn import metrics

import random

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from daf.datasets import atti_dataset
from daf.utils import dataset_utils
from daf.utils import keras_util

In [67]:
num_words = 10000

In [68]:
(x_train, y_train), (x_test, y_test) = atti_dataset.load_data(num_words=num_words)

5537792/5537298 [==============================] - 1s 0us/step


In [69]:
label_index_dict = atti_dataset.get_label_index()

8192/1917 [================================================================================================================================] - 0s 0us/step


In [70]:
max_dimension = max([max(x) for x in x_train])
max_dimension

9999

## Data Preparation

In [71]:
from functools import partial

def x_transformer(x_data):
    return partial(dataset_utils.vectorize_sequences, x_data, num_words)

In [72]:
batch_size = 128
num_classes = len(set(y_train))
train_val_split = math.ceil(len(x_train) * 0.8)

In [73]:
train_val_split

117777

In [74]:
train_generator = dataset_utils.dataset_generator_fun(x_train, y_train, x_transformer, batch_size, 
                                                     0, train_val_split, True)

val_generator = dataset_utils.dataset_generator_fun(x_train, y_train, x_transformer, batch_size,
                                             train_val_split, len(x_train), False)

In [75]:
def build_model(neurons, num_features):
    keras_util.new_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(neurons, activation='relu', input_shape=(num_features, )))
    model.add(tf.keras.layers.Dense(neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(len(label_index_dict), activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [76]:
model = build_model(64, num_words)

In [77]:
model.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                640064    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 34)                2210      
Total params: 650,594
Trainable params: 650,594
Non-trainable params: 0
_________________________________________________________________


In [78]:
train_steps = train_val_split // batch_size + 1
val_steps = (len(x_train) - train_val_split) // batch_size + 1

In [79]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir="logs/dense_512")]

In [80]:
history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs=5, 
                              validation_data=val_generator, validation_steps=val_steps, callbacks=callbacks)

Epoch 1/5
921/921 [==============================] - 29s 32ms/step - loss: 1.3052 - acc: 0.6076 - val_loss: 1.0456 - val_acc: 0.6586
Epoch 2/5
921/921 [==============================] - 29s 32ms/step - loss: 0.7454 - acc: 0.7539 - val_loss: 0.9728 - val_acc: 0.6854
Epoch 3/5
921/921 [==============================] - 29s 32ms/step - loss: 0.6050 - acc: 0.7999 - val_loss: 0.9643 - val_acc: 0.6989
Epoch 4/5
921/921 [==============================] - 29s 32ms/step - loss: 0.5115 - acc: 0.8292 - val_loss: 0.9757 - val_acc: 0.7000
Epoch 5/5
921/921 [==============================] - 27s 30ms/step - loss: 0.4315 - acc: 0.8558 - val_loss: 1.0100 - val_acc: 0.7110


Prediction on the test set

In [ ]:
test_steps = len(x_test) // batch_size + 1
test_generator = data_utils.data_generator(x_test, y_test, dimensions, num_classes, batch_size, 0, len(x_test))

In [ ]:
model.evaluate_generator(test_generator, test_steps)

In [ ]:
test_predictions = model.predict_generator(test_generator, test_steps)

In [ ]:
print(test_predictions.shape)
print(y_test.shape)

In [ ]:
predictions = np.array([np.argmax(x) for x in test_predictions])

In [ ]:
metrics.precision_recall_fscore_support(y_test,predictions, average='weighted')

In [ ]:
metrics.accuracy_score(y_test, predictions)

In [ ]:
y_test_v = data_utils.to_one_hot(y_test, len(set(y_test)))

In [ ]:
metrics.roc_auc_score(y_test_v, test_predictions, average='weighted')

In [ ]:
print(metrics.classification_report(y_test, predictions))

### Confusion Matrix

In [ ]:
conf_matrix = metrics.confusion_matrix(y_test, predictions)

In [ ]:
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
np.set_printoptions(precision=2)

In [ ]:
plt.rcParams['figure.figsize'] = (20,20)
plot_confusion_matrix(conf_matrix, classes=label_index_dict,
                      title='Confusion matrix, without normalization')